In [1]:
link = "https://www.atptour.com/en/scores/stats-centre/archive/2024/301/ms006"


In [1]:
import base64
import json
import string
from datetime import datetime, timezone
from io import StringIO
from typing import Final, IO, TypedDict, Any
from Crypto.Cipher import AES

__all__: tuple[str, ...] = ("main",)


class _Response(TypedDict):
    lastModified: int
    response: str


def _integer_to_string_with_radix(value: int, radix: int) -> str:
    if value < 0:
        message: str = "Negative unsupported."
        raise ValueError(message)

    alphabet: Final[str] = string.digits + string.ascii_letters

    if radix < 2 or radix > len(alphabet):
        message = f"Supported radix range is 2-{len(alphabet)}."
        raise ValueError(message)

    buffer: StringIO
    with StringIO() as buffer:
        while value > 0:
            buffer.write(alphabet[value % radix])
            value //= radix

        return buffer.getvalue()[::-1]


def _get_key_and_iv(last_modified_timestamp: int) -> tuple[bytes, bytes]:
    # Convert to radix 16.
    tmp: int = int(str(last_modified_timestamp), base=16)
    # Reinterpret as radix 36.
    key: str = _integer_to_string_with_radix(tmp, radix=36)

    # Python works with seconds, but in float, so preserve milliseconds.
    last_modified_date: Final[datetime] = datetime.fromtimestamp(last_modified_timestamp / 1000, tz=timezone.utc)
    # Pad with zero if `current_day` less than 10.a
    current_day_reversed: Final[int] = int(f"{last_modified_date.day:02d}"[::-1])
    current_year_reversed: Final[int] = int(str(last_modified_date.year)[::-1])
    # That's funny.
    tmp = (last_modified_date.day + current_day_reversed) * (last_modified_date.year + current_year_reversed)
    key += _integer_to_string_with_radix(tmp, radix=24)

    # Crop to 14 characters and pad with zeros if less.
    key_encoded: Final[bytes] = f"#{key[:14]:014s}$".encode()
    return key_encoded, key_encoded.upper()


def _decrypt(key: bytes, iv: bytes, data: bytes) -> bytes:
    # Strip from padding.
    return AES.new(key, AES.MODE_CBC, iv=iv).decrypt(data).rstrip(b"\x07")


def main() -> None:
    file: IO[str]
    with open(file="./data.example.json", mode="rt") as file:
        response: Final[_Response] = json.load(file)

    key: bytes
    iv: bytes
    key, iv = _get_key_and_iv(response["lastModified"])
    print(key, iv, flush=True)

    encrypted_payload: Final[bytes] = base64.b64decode(response["response"])
    decrypted_payload: Final[bytes] = _decrypt(key, iv, encrypted_payload)
    payload: Final[dict[str, Any]] = json.loads(decrypted_payload)
    print(json.dumps(payload, indent=2), flush=True)

import requests
def from_url(url: str, filename: str) -> None:
    response: Final[_Response] = requests.get(url).json()

    key: bytes
    iv: bytes
    key, iv = _get_key_and_iv(response["lastModified"])
    print(key, iv, flush=True)

    encrypted_payload: Final[bytes] = base64.b64decode(response["response"])
    decrypted_payload: Final[bytes] = _decrypt(key, iv, encrypted_payload)
    payload: Final[dict[str, Any]] = json.loads(remove_padding(decrypted_payload))
    print(json.dumps(payload, indent=2), flush=True)

    with open(filename,'w') as f:
        json.dump(payload, f, indent=2)

def remove_padding(data: bytes) -> bytes:
    # Get the value of the last byte
    padding_len = data[-1]
    # Validate and remove the padding
    if padding_len > 0 and all(byte == padding_len for byte in data[-padding_len:]):
        return data[:-padding_len]
    raise ValueError("Invalid padding in decrypted payload")

# if __name__ == '__main__':
#     main()


In [17]:

response = {
  "lastModified": 1735469923043,
  "response": "...",
}

In [18]:
key, iv = _get_key_and_iv(response["lastModified"])
print(key, iv, flush=True)
encrypted_payload: Final[bytes] = base64.b64decode(response["response"])
decrypted_payload: Final[bytes] = _decrypt(key, iv, encrypted_payload)
payload: Final[dict[str, Any]] = json.loads(remove_padding(decrypted_payload))
print(json.dumps(payload, indent=2), flush=True)



b'#40q1v9mmgz26bl$' b'#40Q1V9MMGZ26BL$'
{
  "courtId": 1,
  "courtName": null,
  "allStats": true,
  "isDoubles": false,
  "eventId": 301,
  "eventType": "Men's Singles",
  "isMatchComplete": false,
  "matchId": "MS006",
  "maxRally": 19,
  "matchStatus": "I",
  "matchWinner": 0,
  "playerData": {
    "tm1Ply1CountryName": "france",
    "tm1Ply1Country": "FRA",
    "tm1Ply1Id": "F0F1",
    "tm1Ply1Name": "A. FILS",
    "tm1Ply1FirstName": "Arthur",
    "tm1Ply1LastName": "FILS",
    "tm1Ply2CountryName": null,
    "tm1Ply2Country": null,
    "tm1Ply2Id": null,
    "tm1Ply2Name": null,
    "tm1Ply2FirstName": null,
    "tm1Ply2LastName": null,
    "tm1Seed": "6",
    "tm1WinProb": 0.0,
    "tm2Ply1CountryName": "germany",
    "tm2Ply1Country": "GER",
    "tm2Ply1Id": "AE14",
    "tm2Ply1Name": "D. ALTMAIER",
    "tm2Ply1FirstName": "Daniel",
    "tm2Ply1LastName": "ALTMAIER",
    "tm2Ply2CountryName": null,
    "tm2Ply2Country": null,
    "tm2Ply2Id": null,
    "tm2Ply2Name": null,
    

In [15]:
decrypted_payload

b'{"a75":false,"a76":"Men\'s Singles","a77":null,"a78":1,"a50":{"1_8_1_1":{"shotHandTypeIds":["NA"],"video":{"videoUrl":null,"thumbnailUrl":null,"duration":null},"a11":"false","a89":3.973799896,"a12":[[{"a101":{"a70":-11.1239996,"a71":-1.31099999,"a72":2.83599997},"a147":{"a70":14.7340002,"a71":3.79399991,"a72":0.0},"a148":{"a70":-11.3459997,"a71":-1.50199997,"a72":0.0},"a149":0.0,"a150":"hit"},{"a101":{"a70":-11.1239996,"a71":-1.31099999,"a72":2.83599997},"a147":{"a70":14.7340002,"a71":3.79399991,"a72":0.0},"a148":{"a70":-11.3459997,"a71":-1.50199997,"a72":0.0},"a149":0.0,"a150":"peak"},{"a101":{"a70":-0.0,"a71":-0.354000002,"a72":1.07700002},"a147":{"a70":14.6420002,"a71":3.91499996,"a72":0.0},"a148":{"a70":-11.0959997,"a71":-1.55200005,"a72":0.0},"a149":0.224000007,"a150":"net"},{"a101":{"a70":5.42399979,"a71":0.180000007,"a72":0.0340000018},"a147":{"a70":14.6990004,"a71":3.98799992,"a72":0.0},"a148":{"a70":-11.1210003,"a71":-1.597,"a72":0.0},"a149":0.351999998,"a150":"bounce"},{"a1

https://www.atptour.com/en/scores/stats-centre/archive/2024/301/ms006?tab=MatchBeats

In [25]:
from_url('https://itp-atp-sls.infosys-platforms.com/prod/api/'
         'match-beats/data/year/2024/eventId/301/matchId/MS006',
         'ms006/match-beats.json')

b'#40q1x42zr426bl$' b'#40Q1X42ZR426BL$'
{
  "courtId": 1,
  "courtName": null,
  "allStats": true,
  "isDoubles": false,
  "eventId": 301,
  "eventType": "Men's Singles",
  "isMatchComplete": false,
  "matchId": "MS006",
  "maxRally": 19,
  "matchStatus": "I",
  "matchWinner": 0,
  "playerData": {
    "tm1Ply1CountryName": "france",
    "tm1Ply1Country": "FRA",
    "tm1Ply1Id": "F0F1",
    "tm1Ply1Name": "A. FILS",
    "tm1Ply1FirstName": "Arthur",
    "tm1Ply1LastName": "FILS",
    "tm1Ply2CountryName": null,
    "tm1Ply2Country": null,
    "tm1Ply2Id": null,
    "tm1Ply2Name": null,
    "tm1Ply2FirstName": null,
    "tm1Ply2LastName": null,
    "tm1Seed": "6",
    "tm1WinProb": 0.0,
    "tm2Ply1CountryName": "germany",
    "tm2Ply1Country": "GER",
    "tm2Ply1Id": "AE14",
    "tm2Ply1Name": "D. ALTMAIER",
    "tm2Ply1FirstName": "Daniel",
    "tm2Ply1LastName": "ALTMAIER",
    "tm2Ply2CountryName": null,
    "tm2Ply2Country": null,
    "tm2Ply2Id": null,
    "tm2Ply2Name": null,
    

In [26]:
from_url('https://itp-atp-sls.infosys-platforms.com/prod/api/'
         'stats-plus/v1/keystats/year/2024/eventId/301/matchId/MS006',
         'ms006/stats-plus.json')


b'#40q1x4zip226bl$' b'#40Q1X4ZIP226BL$'
{
  "courtId": 1,
  "matchCompleted": false,
  "isDoubles": false,
  "setsCompleted": 1,
  "players": [
    {
      "seed": "6",
      "player1Name": "A. FILS",
      "player1Id": "F0F1",
      "player1Country": "FRA"
    },
    {
      "seed": "",
      "player1Name": "D. ALTMAIER",
      "player1Id": "AE14",
      "player1Country": "GER"
    }
  ],
  "setStats": {
    "set0": [
      {
        "order": 1,
        "name": "Serve Rating",
        "player1": "322",
        "player2": "273",
        "player1Bar": 54,
        "player2Bar": 46,
        "influence": "0%",
        "player1Points": [],
        "player2Points": [],
        "player1CrucialPoints": [],
        "player2CrucialPoints": []
      },
      {
        "order": 2,
        "name": "Aces",
        "player1": "0",
        "player2": "2",
        "player1Bar": 0,
        "player2Bar": 100,
        "influence": "1%",
        "player1Points": [],
        "player2Points": [],
        "pl

In [27]:
from_url('https://itp-atp-sls.infosys-platforms.com/prod/api/'
         'rally-analysis/year/2024/eventId/301/matchId/MS006',
         'ms006/rally-analysis.json')

b'#40q1x57zps26bl$' b'#40Q1X57ZPS26BL$'
{
  "setsCompleted": 1,
  "matchCompleted": false,
  "isDoubles": false,
  "maxSets": 2,
  "playerDetails": [
    {
      "seed": "6",
      "player1Name": "A. FILS",
      "player1Id": "F0F1",
      "player1Country": "FRA",
      "player2Name": null,
      "player2Id": null,
      "player2Country": null
    },
    {
      "seed": "",
      "player1Name": "D. ALTMAIER",
      "player1Id": "AE14",
      "player1Country": "GER",
      "player2Name": null,
      "player2Id": null,
      "player2Country": null
    }
  ],
  "pointsMissing": false,
  "rallyData": [
    {
      "name": "Serve",
      "t1err": [],
      "t1win": [
        {
          "crucialPoint": false,
          "score": "0-0",
          "hand": "NA",
          "pointEndType": "UNCLASSIFIED",
          "pointId": "2_2_1_1",
          "serve": 1,
          "serveDir": null,
          "courtSide": null,
          "serveSpeed": 194,
          "set": 2,
          "shotType": null,
      

In [28]:
from_url('https://itp-atp-sls.infosys-platforms.com/prod/api/'
         'stroke-analysis/rally/v2/year/2024/eventId/301/matchId/MS006',
         'ms006/stroke-analysis.json')

b'#40q1x5lgub26bl$' b'#40Q1X5LGUB26BL$'
{
  "courtId": 1,
  "matchCompleted": false,
  "isDoubles": false,
  "setsCompleted": 1,
  "players": [
    {
      "seed": "6",
      "player1Name": "A. FILS",
      "player1Id": "F0F1",
      "player1Country": "FRA",
      "player1Hand": ""
    },
    {
      "seed": "",
      "player1Name": "D. ALTMAIER",
      "player1Id": "AE14",
      "player1Country": "GER",
      "player1Hand": ""
    }
  ],
  "rallyShots": {
    "allPoints": [
      {
        "forehand": [
          {
            "player1": 0,
            "player2": 0,
            "name": "Ground Stroke",
            "player1Wins": 0,
            "player1Frcs": 0,
            "player1Unfs": 0,
            "player1Others": 0,
            "player2Wins": 0,
            "player2Frcs": 0,
            "player2Unfs": 0,
            "player2Others": 0,
            "player1Points": {
              "winners": [],
              "unforcedErrors": [],
              "forcedErrors": [],
              "

In [3]:
from_url('https://itp-atp-sls.infosys-platforms.com/static/prod/rally-analysis/2024/5014/QS042/data.json', 'test.json')

b'#40dw0nhzedekgi$' b'#40DW0NHZEDEKGI$'
{
  "setsCompleted": 2,
  "matchCompleted": true,
  "isDoubles": false,
  "maxSets": 2,
  "playerDetails": [
    {
      "seed": "6",
      "player1Name": "D. SHAPOVALOV",
      "player1Id": "SU55",
      "player1Country": "CAN",
      "player2Name": null,
      "player2Id": null,
      "player2Country": null
    },
    {
      "seed": "",
      "player1Name": "Y. HSU",
      "player1Id": "H09F",
      "player1Country": "TPE",
      "player2Name": null,
      "player2Id": null,
      "player2Country": null
    }
  ],
  "pointsMissing": true,
  "rallyData": [
    {
      "name": "Serve",
      "t1err": [
        {
          "crucialPoint": false,
          "score": "15-0",
          "hand": null,
          "pointEndType": "DOUBLE FAULT",
          "pointId": "1_4_2_2",
          "serve": 2,
          "serveDir": null,
          "courtSide": null,
          "serveSpeed": 0,
          "set": 1,
          "shotType": null,
          "t1BreakPoint": f